In [1]:
import sys
sys.path.insert(0,'../')
from file_tools import *
from request_tools import *
from parse_tools import *
import time
from tqdm import tqdm



SRC_DIR = './data-parsed'
TGT_DIR = './data-computed'


In [23]:
def __compute_days_between_rows__(df,window=2):
    df = pd.to_datetime(df.copy())
    return df.diff(window-1).dt.days # -1 because diff() computes the difference between the current and the previous row

def __compute_cumulative_avg_stats__(df):
    return df.expanding().mean(),df.expanding().std()

def __compute_rolling_avg_stats__(df, window=5):
    return df.rolling(window).mean(),df.rolling(window).std()

def __compute_pts_total__(df):
    return df.sum(axis=1)

def __compute_pts_spread__(df):
    return df.apply(lambda x: x.iloc[0] - x.iloc[1],axis=1)

def __compute_venue_streak_count__(series):
    idx_sequence = []
    count = 0
    for i in range(len(series)):
        if i == 0 or series.iloc[i] != series.iloc[i-1]:
            count = 0
        else:
            count += 1
        idx_sequence.append(count)
    return pd.Series(idx_sequence,index=series.index)

In [24]:
SRC_TGL_CSV_LIST = get_all_files_recursive(f'{SRC_DIR}/teams', file_type=('tgl_basic.csv','tgl_advanced.csv')) # get only regular season gamelog files
SRC_TGL_CSV_LIST = [x for x in SRC_TGL_CSV_LIST if 'BOS/2023' in x] # remove playoff files
SRC_TGL_CSV = SRC_TGL_CSV_LIST[0]
# TQDM_SRC_TGL_CSV_LIST = tqdm(SRC_TGL_CSV_LIST,ncols=150,dynamic_ncols=False)
# for SRC_TGL_CSV in TQDM_SRC_TGL_CSV_LIST:
#     TQDM_SRC_TGL_CSV_LIST.set_description(SRC_TGL_CSV)       
#     # Load the gamelog csv
TGL_DF = pd.read_csv(SRC_TGL_CSV,header=[0,1])
TGL_DF = TGL_DF.drop(
    columns=[
        ('Unnamed: 0_level_0','Unnamed: 0_level_1')
    ]).rename(
    columns={
        'Boxscores_html_id':'Boxscores_id',
        'Tm_html_id':'Team_id',
        'Opp_html_id':'Opp_id',
    },level=1
)
TGL_IDX = pd.MultiIndex.from_arrays([TGL_DF.index,TGL_DF[('Match','Team_id')]],names=['index','Team_id'])
TGL_DF.set_index(TGL_IDX,inplace=True)
# Compute the expanding and rolling stats
TGL_MDATE_DF = TGL_DF[('Match','Date')]
TM_HM_AW = TGL_DF[('Match','H/A')]
TGL_VEN_STRK_CNT = __compute_venue_streak_count__(TGL_DF[('Match','H/A')]).rename(('Match','Venue_Strk_Cnt'))
TGL_DAYS_BTW_2GM = __compute_days_between_rows__(TGL_MDATE_DF,window=2).rename(('Match','Days_Btw_2GM'))
TGL_DAYS_BTW_3GM = __compute_days_between_rows__(TGL_MDATE_DF,window=3).rename(('Match','Days_Btw_3GM'))
TGL_DAYS_BTW_4GM = __compute_days_between_rows__(TGL_MDATE_DF,window=4).rename(('Match','Days_Btw_4GM'))
TGL_DAYS_BTW_GM_DF = pd.concat([TM_HM_AW,TGL_VEN_STRK_CNT,TGL_DAYS_BTW_2GM,TGL_DAYS_BTW_3GM,TGL_DAYS_BTW_4GM],axis=1) 
# Compute the total points, spread points
TGL_RESULT_DF = TGL_DF['Result'].copy()
TGL_PTS_TOTAL = __compute_pts_total__(TGL_RESULT_DF[['Tm','Opp']]).rename('Pts_Total')
TGL_PTS_SPREAD = __compute_pts_spread__(TGL_RESULT_DF[['Tm','Opp']]).rename('Pts_Spread')
TGL_RESULT_DF = pd.concat([TGL_RESULT_DF,TGL_PTS_TOTAL,TGL_PTS_SPREAD],axis=1)
# Compute the expanding and rolling stats
TGL_STATS_DF = TGL_DF.drop('Match',level=0,axis=1)
TGL_STATS_DF

Result           Advanced                        
                              W/L   Tm  Opp     ORtg   DRtg   Pace    FTr   
index Team_id                                                               
0     /teams/BOS/2023.html      1  126  117    128.3  119.2   98.2  0.341  \
1     /teams/BOS/2023.html      1  111  104    120.8  113.2   91.9  0.190   
2     /teams/BOS/2023.html      1  126  120    128.4  122.3   98.1  0.322   
3     /teams/BOS/2023.html      0  102  120    104.2  122.6   97.8  0.120   
4     /teams/BOS/2023.html      0  123  132    115.4  123.9   96.5  0.390   
...                           ...  ...  ...      ...    ...    ...    ...   
77    /teams/BOS/2023.html      1  122  114    117.2  109.5  104.1  0.229   
78    /teams/BOS/2023.html      0  101  103    114.7  117.0   88.0  0.176   
79    /teams/BOS/2023.html      1   97   93    101.2   97.0   95.9  0.202   
80    /teams/BOS/2023.html      1  121  102    128.6  108.4   94.1  0.259   
81    /teams/BOS/2023.html      1  120  114    124.8  118.6   96.1  0.157   

                                                ...              
                             3PAr    TS%  TRB%  ... STL%  BLK%   
index Team_id                                   ...              
0     /teams/BOS/2023.html  0.427  0.668  53.7  ...  8.1   6.5  \
1     /teams/BOS/2023.html  0.418  0.648  54.4  ...  4.4  16.7   
2     /teams/BOS/2023.html  0.540  0.634  43.5  ...  5.1   9.8   
3     /teams/BOS/2023.html  0.480  0.484  42.9  ...  5.1  10.6   
4     /teams/BOS/2023.html  0.402  0.640  44.7  ...  8.4   3.8   
...                           ...    ...   ...  ...  ...   ...   
77    /teams/BOS/2023.html  0.531  0.577  44.7  ...  7.7   6.0   
78    /teams/BOS/2023.html  0.495  0.515  53.8  ...  4.5   8.2   
79    /teams/BOS/2023.html  0.427  0.500  49.0  ...  5.2  12.3   
80    /teams/BOS/2023.html  0.543  0.670  60.2  ...  4.3  11.3   
81    /teams/BOS/2023.html  0.607  0.631  48.5  ...  5.2   5.2   

                           Offensive Four Factors                      
                                             eFG%  TOV%  ORB% FT/FGA   
index Team_id                                                          
0     /teams/BOS/2023.html                  0.634   9.6  18.2  0.268  \
1     /teams/BOS/2023.html                  0.608  18.2  32.4  0.190   
2     /teams/BOS/2023.html                  0.592   7.5  16.3  0.264   
3     /teams/BOS/2023.html                  0.460   8.7  23.3  0.100   
4     /teams/BOS/2023.html                  0.591  16.5  13.2  0.317   
...                                           ...   ...   ...    ...   
77    /teams/BOS/2023.html                  0.526   9.4  20.4  0.219   
78    /teams/BOS/2023.html                  0.500   7.5  29.5  0.110   
79    /teams/BOS/2023.html                  0.489  13.4  22.9  0.112   
80    /teams/BOS/2023.html                  0.648  17.4  36.1  0.198   
81    /teams/BOS/2023.html                  0.612  13.6  26.1  0.124   

                           Defensive Four Factors                     
                                             eFG%  TOV%  DRB% FT/FGA  
index Team_id                                                         
0     /teams/BOS/2023.html                  0.581  13.2  88.2  0.300  
1     /teams/BOS/2023.html                  0.500   8.4  73.8  0.130  
2     /teams/BOS/2023.html                  0.553   7.9  71.4  0.158  
3     /teams/BOS/2023.html                  0.540  11.5  68.9  0.299  
4     /teams/BOS/2023.html                  0.609  12.8  76.3  0.299  
...                                           ...   ...   ...    ...  
77    /teams/BOS/2023.html                  0.526  15.0  71.1  0.135  
78    /teams/BOS/2023.html                  0.571  10.5  83.3  0.179  
79    /teams/BOS/2023.html                  0.444  10.1  74.0  0.144  
80    /teams/BOS/2023.html                  0.478   6.5  76.9  0.140  
81    /teams/BOS/2023.html                  0.510   7.9  68.6  0.155  

[82 rows x 21 colum

In [28]:
# Compute the total points, spread points
TGL_RESULT_DF = TGL_DF['Result'].copy()
TGL_PTS_TOTAL = __compute_pts_total__(TGL_RESULT_DF[['Tm','Opp']]).rename('Pts_Total')
TGL_PTS_SPREAD = __compute_pts_spread__(TGL_RESULT_DF[['Tm','Opp']]).rename('Pts_Spread')
TGL_RESULT_DF = pd.concat([TGL_RESULT_DF,TGL_PTS_TOTAL,TGL_PTS_SPREAD],axis=1)
# Compute the expanding and rolling stats
TGL_STATS_DF = TGL_DF.drop('Match',level=0,axis=1)
TGL_STATS_CUMU_AVG_DF,TGL_STATS_CUMU_STD_DF        = __compute_cumulative_avg_stats__(TGL_STATS_DF)
TGL_STATS_ROLL_04_AVG_DF, TGL_STATS_ROLL_04_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_DF,window=4)
# TGL_STATS_ROLL_08_AVG_DF, TGL_STATS_ROLL_08_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_DF,window=8)
# TGL_STATS_ROLL_12_AVG_DF, TGL_STATS_ROLL_12_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_DF,window=12)
# TGL_STATS_ROLL_16_AVG_DF, TGL_STATS_ROLL_16_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_DF,window=16)
# TGL_STATS_ROLL_20_AVG_DF, TGL_STATS_ROLL_20_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_DF,window=20)
TGL_STATS_CUMU_AVG_DF

Result                            Advanced   
                                 W/L          Tm         Opp        ORtg   
index Team_id                                                              
0     /teams/BOS/2023.html  1.000000  126.000000  117.000000  128.300000  \
1     /teams/BOS/2023.html  1.000000  118.500000  110.500000  124.550000   
2     /teams/BOS/2023.html  1.000000  121.000000  113.666667  125.833333   
3     /teams/BOS/2023.html  0.750000  116.250000  115.250000  120.425000   
4     /teams/BOS/2023.html  0.600000  117.600000  118.600000  119.420000   
...                              ...         ...         ...         ...   
77    /teams/BOS/2023.html  0.692308  118.358974  111.846154  119.062821   
78    /teams/BOS/2023.html  0.683544  118.139241  111.734177  119.007595   
79    /teams/BOS/2023.html  0.687500  117.875000  111.500000  118.785000   
80    /teams/BOS/2023.html  0.691358  117.913580  111.382716  118.906173   
81    /teams/BOS/2023.html  0.695122  117.939024  111.414634  118.978049   

                                                                        
                                  DRtg       Pace       FTr      3PAr   
index Team_id                                                           
0     /teams/BOS/2023.html  119.200000  98.200000  0.341000  0.427000  \
1     /teams/BOS/2023.html  116.200000  95.050000  0.265500  0.422500   
2     /teams/BOS/2023.html  118.233333  96.066667  0.284333  0.461667   
3     /teams/BOS/2023.html  119.325000  96.500000  0.243250  0.466250   
4     /teams/BOS/2023.html  120.240000  96.500000  0.272600  0.453400   
...                                ...        ...       ...       ...   
77    /teams/BOS/2023.html  112.379487  98.024359  0.248308  0.478090   
78    /teams/BOS/2023.html  112.437975  97.897468  0.247392  0.478304   
79    /teams/BOS/2023.html  112.245000  97.872500  0.246825  0.477662   
80    /teams/BOS/2023.html  112.197531  97.825926  0.246975  0.478469   
81    /teams/BOS/2023.html  112.275610  97.804878  0.245878  0.480037   

                                                 ...                        
                                 TS%       TRB%  ...      STL%       BLK%   
index Team_id                                    ...                        
0     /teams/BOS/2023.html  0.668000  53.700000  ...  8.100000   6.500000  \
1     /teams/BOS/2023.html  0.658000  54.050000  ...  6.250000  11.600000   
2     /teams/BOS/2023.html  0.650000  50.533333  ...  5.866667  11.000000   
3     /teams/BOS/2023.html  0.608500  48.625000  ...  5.675000  10.900000   
4     /teams/BOS/2023.html  0.614800  47.840000  ...  6.220000   9.480000   
...                              ...        ...  ...       ...        ...   
77    /teams/BOS/2023.html  0.602923  50.679487  ...  6.428205   9.226923   
78    /teams/BOS/2023.html  0.601810  50.718987  ...  6.403797   9.213924   
79    /teams/BOS/2023.html  0.600537  50.697500  ...  6.388750   9.252500   
80    /teams/BOS/2023.html  0.601395  50.814815  ...  6.362963   9.277778   
81    /teams/BOS/2023.html  0.601756  50.786585  ...  6.348780   9.228049   

                           Offensive Four Factors                         
                                             eFG%       TOV%       ORB%   
index Team_id                                                             
0     /teams/BOS/2023.html               0.634000   9.600000  18.200000  \
1     /teams/BOS/2023.html               0.621000  13.900000  25.300000   
2     /teams/BOS/2023.html               0.611333  11.766667  22.300000   
3     /teams/BOS/2023.html               0.573500  11.000000  22.550000   
4     /teams/BOS/2023.html               0.577000  12.100000  20.680000   
...                                           ...        ...        ...   
77    /teams/BOS/2023.html               0.566987  11.375641  21.521795   
78    /teams/BOS/2023.html               0.566139  11.326582  21.622785   
79    /teams/BOS/2023.html            

In [27]:
TGL_STATS_AW_DF = TGL_DF[TGL_DF[('Match','H/A')] == 0].drop('Match',level=0,axis=1)
TGL_STATS_HM_DF = TGL_DF[TGL_DF[('Match','H/A')] == 1].drop('Match',level=0,axis=1)
TGL_STATS_HM_DF_AVG, TGL_STATS_HM_DF_STD = __compute_rolling_avg_stats__(TGL_STATS_HM_DF)
TGL_STATS_HM_DF_AVG

Result               Advanced                   
                              W/L     Tm    Opp     ORtg    DRtg    Pace   
index Team_id                                                              
0     /teams/BOS/2023.html    NaN    NaN    NaN      NaN     NaN     NaN  \
4     /teams/BOS/2023.html    NaN    NaN    NaN      NaN     NaN     NaN   
5     /teams/BOS/2023.html    NaN    NaN    NaN      NaN     NaN     NaN   
7     /teams/BOS/2023.html    NaN    NaN    NaN      NaN     NaN     NaN   
10    /teams/BOS/2023.html    0.8  122.4  114.8   121.56  113.66   98.78   
11    /teams/BOS/2023.html    0.8  123.4  113.8   124.08  113.92   97.74   
13    /teams/BOS/2023.html    1.0  124.0  111.8   123.44  110.86  100.90   
17    /teams/BOS/2023.html    1.0  126.6  115.4   125.60  114.14  101.32   
18    /teams/BOS/2023.html    1.0  126.4  112.4   126.32  112.04  100.54   
19    /teams/BOS/2023.html    1.0  126.8  114.2   128.62  115.64   99.20   
20    /teams/BOS/2023.html    1.0  128.6  112.8   129.08  113.02  100.14   
21    /teams/BOS/2023.html    1.0  130.2  112.6   134.22  116.20   97.12   
22    /teams/BOS/2023.html    0.8  128.4  114.2   129.54  114.90   97.66   
29    /teams/BOS/2023.html    0.6  125.8  116.8   126.90  117.44   97.70   
30    /teams/BOS/2023.html    0.4  118.2  111.6   117.20  110.28   99.18   
31    /teams/BOS/2023.html    0.2  112.6  114.0   110.82  112.04   99.78   
32    /teams/BOS/2023.html    0.2  110.0  111.6   107.48  108.98  100.30   
33    /teams/BOS/2023.html    0.4  114.6  111.2   114.20  110.80  100.34   
34    /teams/BOS/2023.html    0.6  118.0  108.2   117.96  108.12  100.04   
35    /teams/BOS/2023.html    0.8  122.8  111.2   123.16  111.50   99.70   
40    /teams/BOS/2023.html    1.0  121.8  107.6   123.90  109.54   98.16   
41    /teams/BOS/2023.html    1.0  122.6  108.6   125.32  111.10   97.68   
45    /teams/BOS/2023.html    1.0  119.0  108.6   118.68  108.12   98.44   
49    /teams/BOS/2023.html    0.8  117.2  112.2   115.92  110.78   97.32   
50    /teams/BOS/2023.html    0.8  119.0  114.4   115.52  110.86   97.18   
51    /teams/BOS/2023.html    0.8  125.4  113.8   122.00  109.94   97.44   
52    /teams/BOS/2023.html    0.6  119.2  112.2   115.60  108.26   97.48   
54    /teams/BOS/2023.html    0.6  116.2  108.4   118.70  110.28   94.20   
55    /teams/BOS/2023.html    0.8  118.2  107.6   120.30  109.20   96.44   
56    /teams/BOS/2023.html    0.8  117.0  105.2   122.30  109.82   95.82   
58    /teams/BOS/2023.html    0.8  114.6  107.8   118.86  111.82   96.46   
62    /teams/BOS/2023.html    1.0  119.2  109.2   124.46  114.10   95.78   
63    /teams/BOS/2023.html    0.8  119.0  112.4   120.94  114.14   98.58   
64    /teams/BOS/2023.html    0.6  119.4  115.4   119.16  114.80   96.64   
66    /teams/BOS/2023.html    0.6  118.6  112.2   116.78  110.16   97.90   
73    /teams/BOS/2023.html    0.6  117.2  109.4   113.80  106.06   99.20   
74    /teams/BOS/2023.html    0.6  121.2  105.4   114.36   99.32  101.92   
77    /teams/BOS/2023.html    0.8  124.6  105.2   117.22   98.68  102.34   
79    /teams/BOS/2023.html    1.0  118.2   97.6   115.02   95.30  102.48   
80    /teams/BOS/2023.html    1.0  119.4   99.4   117.88   98.50  101.18   
81    /teams/BOS/2023.html    1.0  119.4  103.2   119.78  103.96   99.60   

                                                           ...                
                               FTr    3PAr     TS%   TRB%  ...  STL%   BLK%   
index Team_id                                              ...                
0     /teams/BOS/2023.html     NaN     NaN     NaN    NaN  ...   NaN    NaN  \
4     /teams/BOS/2023.html     NaN     NaN     NaN    NaN  ...   NaN    NaN   
5     /teams/BOS/2023.html     NaN     NaN     NaN    NaN  ...   NaN    NaN   
7     /teams/BOS/2023.html     NaN     NaN     NaN    NaN  ...   NaN    NaN   
10    /teams/BOS/2023.html  0.3386  0.4640  0.6436  49.00  ...  6.10   7.54   
11    /teams/BOS/2023.html  0.3148  0.4608  0.6426

In [11]:


TGL_STATS_H2H_CUMU_AVG_DF, TGL_STATS_H2H_CUMU_STD_DF = __compute_h2h_avg_stats__(TGL_DF)
TGL_STATS_H2H_CUMU_AVG_DF


Result                            Advanced   
                              W/L          Tm         Opp        ORtg   
index Team_id                                                           
0     /teams/BOS/2023.html   1.00  126.000000  117.000000  128.300000  \
1     /teams/BOS/2023.html   1.00  111.000000  104.000000  120.800000   
2     /teams/BOS/2023.html   1.00  126.000000  120.000000  128.400000   
3     /teams/BOS/2023.html   0.00  102.000000  120.000000  104.200000   
4     /teams/BOS/2023.html   0.00  123.000000  132.000000  115.400000   
...                           ...         ...         ...         ...   
77    /teams/BOS/2023.html   0.50  119.500000  116.000000  120.700000   
78    /teams/BOS/2023.html   0.75  110.750000  106.500000  120.350000   
79    /teams/BOS/2023.html   1.00  106.333333  102.333333  110.966667   
80    /teams/BOS/2023.html   1.00  110.000000  102.250000  115.375000   
81    /teams/BOS/2023.html   1.00  126.666667  113.333333  127.666667   

                                                                                
                                  DRtg       Pace       FTr    3PAr       TS%   
index Team_id                                                                   
0     /teams/BOS/2023.html  119.200000  98.200000  0.341000  0.4270  0.668000  \
1     /teams/BOS/2023.html  113.200000  91.900000  0.190000  0.4180  0.648000   
2     /teams/BOS/2023.html  122.300000  98.100000  0.322000  0.5400  0.634000   
3     /teams/BOS/2023.html  122.600000  97.800000  0.120000  0.4800  0.484000   
4     /teams/BOS/2023.html  123.900000  96.500000  0.390000  0.4020  0.640000   
...                                ...        ...       ...     ...       ...   
77    /teams/BOS/2023.html  117.400000  99.150000  0.205500  0.5555  0.596500   
78    /teams/BOS/2023.html  115.850000  91.875000  0.197250  0.4505  0.611250   
79    /teams/BOS/2023.html  106.800000  95.800000  0.203667  0.4130  0.554333   
80    /teams/BOS/2023.html  107.200000  95.375000  0.217500  0.4455  0.583250   
81    /teams/BOS/2023.html  114.266667  99.166667  0.132333  0.5240  0.660333   

                                   ...                        
                             TRB%  ...      STL%       BLK%   
index Team_id                      ...                        
0     /teams/BOS/2023.html  53.70  ...  8.100000   6.500000  \
1     /teams/BOS/2023.html  54.40  ...  4.400000  16.700000   
2     /teams/BOS/2023.html  43.50  ...  5.100000   9.800000   
3     /teams/BOS/2023.html  42.90  ...  5.100000  10.600000   
4     /teams/BOS/2023.html  44.70  ...  8.400000   3.800000   
...                           ...  ...       ...        ...   
77    /teams/BOS/2023.html  43.20  ...  4.900000   4.800000   
78    /teams/BOS/2023.html  53.25  ...  5.325000   8.025000   
79    /teams/BOS/2023.html  50.20  ...  6.966667   9.833333   
80    /teams/BOS/2023.html  52.70  ...  6.300000  10.200000   
81    /teams/BOS/2023.html  48.70  ...  5.033333   8.633333   

                           Offensive Four Factors                      
                                             eFG%    TOV%       ORB%   
index Team_id                                                          
0     /teams/BOS/2023.html               0.634000   9.600  18.200000  \
1     /teams/BOS/2023.html               0.608000  18.200  32.400000   
2     /teams/BOS/2023.html               0.592000   7.500  16.300000   
3     /teams/BOS/2023.html               0.460000   8.700  23.300000   
4     /teams/BOS/2023.html               0.591000  16.500  13.200000   
...                                           ...     ...        ...   
77    /teams/BOS/2023.html               0.564000   7.650  15.900000   
78    /teams/BOS/2023.html               0.594500  10.650  20.825000   
79    /teams/BOS/2023.html               0.530333  12.500  24.833333   
80    /teams/BOS/2023.html               0.559750  13.725  27.650000   
81    /teams/BOS/2023.html               0.643333 

In [12]:
TGL_STATS_H2H_CUMU_STD_DF

Result                         Advanced   
                                 W/L         Tm        Opp       ORtg   
index Team_id                                                           
0     /teams/BOS/2023.html       NaN        NaN        NaN        NaN  \
1     /teams/BOS/2023.html       NaN        NaN        NaN        NaN   
2     /teams/BOS/2023.html       NaN        NaN        NaN        NaN   
3     /teams/BOS/2023.html       NaN        NaN        NaN        NaN   
4     /teams/BOS/2023.html       NaN        NaN        NaN        NaN   
...                              ...        ...        ...        ...   
77    /teams/BOS/2023.html  0.707107   3.535534   2.828427   4.949747   
78    /teams/BOS/2023.html  0.500000  10.812801   7.724420   5.806605   
79    /teams/BOS/2023.html  0.000000   9.504385   8.621678   9.377811   
80    /teams/BOS/2023.html  0.000000  10.677078   7.041543  11.677436   
81    /teams/BOS/2023.html  0.000000   7.023769  12.013881   3.384277   

                                                                                
                                 DRtg      Pace       FTr      3PAr       TS%   
index Team_id                                                                   
0     /teams/BOS/2023.html        NaN       NaN       NaN       NaN       NaN  \
1     /teams/BOS/2023.html        NaN       NaN       NaN       NaN       NaN   
2     /teams/BOS/2023.html        NaN       NaN       NaN       NaN       NaN   
3     /teams/BOS/2023.html        NaN       NaN       NaN       NaN       NaN   
4     /teams/BOS/2023.html        NaN       NaN       NaN       NaN       NaN   
...                               ...       ...       ...       ...       ...   
77    /teams/BOS/2023.html  11.172287  7.000357  0.033234  0.034648  0.027577   
78    /teams/BOS/2023.html   2.891943  4.901275  0.097725  0.034809  0.070315   
79    /teams/BOS/2023.html   8.719518  0.953939  0.028537  0.013528  0.047120   
80    /teams/BOS/2023.html   7.164263  1.152895  0.036171  0.065932  0.069462   
81    /teams/BOS/2023.html  11.150486  2.956913  0.022189  0.082505  0.026350   

                                      ...                       
                                TRB%  ...      STL%      BLK%   
index Team_id                         ...                       
0     /teams/BOS/2023.html       NaN  ...       NaN       NaN  \
1     /teams/BOS/2023.html       NaN  ...       NaN       NaN   
2     /teams/BOS/2023.html       NaN  ...       NaN       NaN   
3     /teams/BOS/2023.html       NaN  ...       NaN       NaN   
4     /teams/BOS/2023.html       NaN  ...       NaN       NaN   
...                              ...  ...       ...       ...   
77    /teams/BOS/2023.html  2.121320  ...  3.959798  1.697056   
78    /teams/BOS/2023.html  1.391642  ...  2.495830  1.276388   
79    /teams/BOS/2023.html  1.311488  ...  2.250185  3.931073   
80    /teams/BOS/2023.html  5.113381  ...  2.270095  3.292416   
81    /teams/BOS/2023.html  3.304542  ...  0.960902  3.225420   

                           Offensive Four Factors                       
                                             eFG%      TOV%      ORB%   
index Team_id                                                           
0     /teams/BOS/2023.html                    NaN       NaN       NaN  \
1     /teams/BOS/2023.html                    NaN       NaN       NaN   
2     /teams/BOS/2023.html                    NaN       NaN       NaN   
3     /teams/BOS/2023.html                    NaN       NaN       NaN   
4     /teams/BOS/2023.html                    NaN       NaN       NaN   
...                                           ...       ...       ...   
77    /teams/BOS/2023.html               0.053740  2.474874  6.363961   
78    /teams/BOS/2023.html               0.067540  2.738004  6.315787   
79    /teams/BOS/2023.html               0.037166  2.193171  2.532456   
80    /teams/BOS/2023.html               0.066199  3.034661  6.000833   
81    /teams

In [ ]:
# Convert this into a function that can be applied to a series. 
# Compute consecutive home/away count


In [ ]:
def __compute_days_between_rows__(df,window=2):
    df = pd.to_datetime(df.copy())
    return df.diff(window-1).dt.days # -1 because diff() computes the difference between the current and the previous row

def __compute_cumulative_avg_stats__(df):
    return df.expanding().mean(),df.expanding().std()

def __compute_rolling_avg_stats__(df, window=5):
    return df.rolling(window).mean(),df.rolling(window).std()

def __compute_pts_total__(df):
    return df.sum(axis=1)

def __compute_pts_spread__(df):
    return df.apply(lambda x: x.iloc[0] - x.iloc[1],axis=1)


def compute_gamelog_stats_regular_season():
    """
    Description:
        Compute gamelog stats
    Summary:
        1. Load all gamelog csv
        2. Compute the expanding and rolling stats
        3. Save the computed gamelogs csv
    Usage:
        python3 03_compute_script.py -m gamelogs_stats -s ../data-parsed/ -t ../data-computed/
    
    """
    _FAILS_ = []
    # Get the list of computed gamelog files
    # TGT_TGL_CSV_COMP_TXT = f'{TGT_DIR}/tgl_csv_stats_computed.txt'
    # TGT_TGL_CSV_COMP_LIST = []
    # if file_exists(TGT_TGL_CSV_COMP_TXT):
    #     TGT_TGL_CSV_COMP_LIST = load_file(TGT_TGL_CSV_COMP_TXT).split('\n')

    # Get all gamelog files
    SRC_TGL_CSV_LIST = get_all_files_recursive(f'{SRC_DIR}/teams', file_type=('tgl_basic.csv','tgl_advanced.csv')) # get only regular season gamelog files
    if debug:
        SRC_TGL_CSV_LIST = SRC_TGL_CSV_LIST[:300]

    TQDM_SRC_TGL_CSV_LIST = tqdm(SRC_TGL_CSV_LIST,ncols=150,dynamic_ncols=False)
    for SRC_TGL_CSV in TQDM_SRC_TGL_CSV_LIST:
        TQDM_SRC_TGL_CSV_LIST.set_description(SRC_TGL_CSV)       
        try:
            # Load the gamelog csv
            TGL_DF = pd.read_csv(SRC_TGL_CSV,header=[0,1])
            TGL_DF = TGL_DF.drop(
                columns=[
                    ('Unnamed: 0_level_0','Unnamed: 0_level_1')
                ]).rename(
                columns={
                    'Boxscores_html_id':'Boxscores_id',
                    'Tm_html_id':'Team_id',
                    'Opp_html_id':'Opp_id',
                },level=1
            )
            TGL_IDX = pd.MultiIndex.from_arrays([TGL_DF.index,TGL_DF[('Match','Team_id')]],names=['index','Team_id'])
            TGL_DF.set_index(TGL_IDX,inplace=True)
            
            # Compute the expanding and rolling stats
            TGL_MDATE_DF = TGL_DF[('Match','Date')]
            TM_HM_AW = TGL_DF[('Match','H/A')]
            TGL_VEN_STRK_CNT = __compute_venue_streak_count__(TGL_DF[('Match','H/A')]).rename(('Match','Venue_Strk_Cnt'))
            TGL_DAYS_BTW_2GM = __compute_days_between_rows__(TGL_MDATE_DF,window=2).rename(('Match','Days_Btw_2GM'))
            TGL_DAYS_BTW_3GM = __compute_days_between_rows__(TGL_MDATE_DF,window=3).rename(('Match','Days_Btw_3GM'))
            TGL_DAYS_BTW_4GM = __compute_days_between_rows__(TGL_MDATE_DF,window=4).rename(('Match','Days_Btw_4GM'))
            TGL_DAYS_BTW_GM_DF = pd.concat([TM_HM_AW,TGL_VEN_STRK_CNT,TGL_DAYS_BTW_2GM,TGL_DAYS_BTW_3GM,TGL_DAYS_BTW_4GM],axis=1) 
            # Compute the total points, spread points
            TGL_RESULT_DF = TGL_DF['Result'].copy()
            TGL_PTS_TOTAL = __compute_pts_total__(TGL_RESULT_DF[['Tm','Opp']]).rename('Pts_Total')
            TGL_PTS_SPREAD = __compute_pts_spread__(TGL_RESULT_DF[['Tm','Opp']]).rename('Pts_Spread')
            TGL_RESULT_DF = pd.concat([TGL_RESULT_DF,TGL_PTS_TOTAL,TGL_PTS_SPREAD],axis=1)
            # Compute the expanding and rolling stats
            TGL_STATS_DF = TGL_DF.drop('Match',level=0,axis=1)
            TGL_STATS_CUMU_AVG_DF,TGL_STATS_CUMU_STD_DF        = __compute_cumulative_avg_stats__(TGL_STATS_DF)
            TGL_STATS_ROLL_04_AVG_DF, TGL_STATS_ROLL_04_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_DF,window=4)
            TGL_STATS_ROLL_08_AVG_DF, TGL_STATS_ROLL_08_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_DF,window=8)
            TGL_STATS_ROLL_12_AVG_DF, TGL_STATS_ROLL_12_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_DF,window=12)
            TGL_STATS_ROLL_16_AVG_DF, TGL_STATS_ROLL_16_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_DF,window=16)
            TGL_STATS_ROLL_20_AVG_DF, TGL_STATS_ROLL_20_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_DF,window=20)
            # Compute the home/away stats
            TGL_STATS_AW_DF = TGL_DF[TGL_DF[('Match','H/A')] == 0].drop('Match',level=0,axis=1)
            TGL_STATS_HM_DF = TGL_DF[TGL_DF[('Match','H/A')] == 1].drop('Match',level=0,axis=1)
            TGL_STATS_AW_CUMU_AVG_DF, TGL_STATS_AW_CUMU_STD_DF      = __compute_cumulative_avg_stats__(TGL_STATS_AW_DF)
            TGL_STATS_AW_ROLL_04_AVG_DF,TGL_STATS_AW_ROLL_04_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_AW_DF,window=4)
            TGL_STATS_AW_ROLL_08_AVG_DF,TGL_STATS_AW_ROLL_08_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_AW_DF,window=8)
            TGL_STATS_AW_ROLL_12_AVG_DF,TGL_STATS_AW_ROLL_12_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_AW_DF,window=12)
            TGL_STATS_AW_ROLL_16_AVG_DF,TGL_STATS_AW_ROLL_16_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_AW_DF,window=16)
            TGL_STATS_AW_ROLL_20_AVG_DF,TGL_STATS_AW_ROLL_20_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_AW_DF,window=20)
            TGL_STATS_HM_CUMU_AVG_DF, TGL_STATS_HM_CUMU_STD_DF      = __compute_cumulative_avg_stats__(TGL_STATS_HM_DF)
            TGL_STATS_HM_ROLL_04_AVG_DF,TGL_STATS_HM_ROLL_04_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_HM_DF,window=4)
            TGL_STATS_HM_ROLL_08_AVG_DF,TGL_STATS_HM_ROLL_08_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_HM_DF,window=8)
            TGL_STATS_HM_ROLL_12_AVG_DF,TGL_STATS_HM_ROLL_12_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_HM_DF,window=12)
            TGL_STATS_HM_ROLL_16_AVG_DF,TGL_STATS_HM_ROLL_16_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_HM_DF,window=16)
            TGL_STATS_HM_ROLL_20_AVG_DF,TGL_STATS_HM_ROLL_20_STD_DF = __compute_rolling_avg_stats__(TGL_STATS_HM_DF,window=20)
            # Compute the h2h stats
            TGL_STATS_H2H_CUMU_AVG_DF, TGL_STATS_H2H_CUMU_STD_DF    = __compute_h2h_avg_stats__(TGL_DF)

            # Save the computed gamelogs csv
            TGL_DICT = {
                'mday_rest_days' : TGL_DAYS_BTW_GM_DF,   'mday_gm_results'   : TGL_RESULT_DF,
                'stats_cumu_avg'    : TGL_STATS_CUMU_AVG_DF,    'stats_cumu_std'    : TGL_STATS_CUMU_STD_DF,
                'stats_roll_04_avg' : TGL_STATS_ROLL_04_AVG_DF, 'stats_roll_04_std' : TGL_STATS_ROLL_04_STD_DF,
                'stats_roll_08_avg' : TGL_STATS_ROLL_08_AVG_DF, 'stats_roll_08_std' : TGL_STATS_ROLL_08_STD_DF,
                'stats_roll_12_avg' : TGL_STATS_ROLL_12_AVG_DF, 'stats_roll_12_std' : TGL_STATS_ROLL_12_STD_DF,
                'stats_roll_16_avg' : TGL_STATS_ROLL_16_AVG_DF, 'stats_roll_16_std' : TGL_STATS_ROLL_16_STD_DF,
                'stats_roll_20_avg' : TGL_STATS_ROLL_20_AVG_DF, 'stats_roll_20_std' : TGL_STATS_ROLL_20_STD_DF,
                'stats_hm_cumu_avg' : TGL_STATS_HM_CUMU_AVG_DF, 'stats_hm_cumu_std' : TGL_STATS_HM_CUMU_STD_DF,
                'stats_hm_roll_04_avg' : TGL_STATS_HM_ROLL_04_AVG_DF, 'stats_hm_roll_04_std' : TGL_STATS_HM_ROLL_04_STD_DF,
                'stats_hm_roll_08_avg' : TGL_STATS_HM_ROLL_08_AVG_DF, 'stats_hm_roll_08_std' : TGL_STATS_HM_ROLL_08_STD_DF,
                'stats_hm_roll_12_avg' : TGL_STATS_HM_ROLL_12_AVG_DF, 'stats_hm_roll_12_std' : TGL_STATS_HM_ROLL_12_STD_DF,
                'stats_hm_roll_16_avg' : TGL_STATS_HM_ROLL_16_AVG_DF, 'stats_hm_roll_16_std' : TGL_STATS_HM_ROLL_16_STD_DF,
                'stats_hm_roll_20_avg' : TGL_STATS_HM_ROLL_20_AVG_DF, 'stats_hm_roll_20_std' : TGL_STATS_HM_ROLL_20_STD_DF,
                'stats_aw_cumu_avg' : TGL_STATS_AW_CUMU_AVG_DF, 'stats_aw_cumu_std' : TGL_STATS_AW_CUMU_STD_DF,
                'stats_aw_roll_04_avg' : TGL_STATS_AW_ROLL_04_AVG_DF, 'stats_aw_roll_04_std' : TGL_STATS_AW_ROLL_04_STD_DF,
                'stats_aw_roll_08_avg' : TGL_STATS_AW_ROLL_08_AVG_DF, 'stats_aw_roll_08_std' : TGL_STATS_AW_ROLL_08_STD_DF,
                'stats_aw_roll_12_avg' : TGL_STATS_AW_ROLL_12_AVG_DF, 'stats_aw_roll_12_std' : TGL_STATS_AW_ROLL_12_STD_DF,
                'stats_aw_roll_16_avg' : TGL_STATS_AW_ROLL_16_AVG_DF, 'stats_aw_roll_16_std' : TGL_STATS_AW_ROLL_16_STD_DF,
                'stats_aw_roll_20_avg' : TGL_STATS_AW_ROLL_20_AVG_DF, 'stats_aw_roll_20_std' : TGL_STATS_AW_ROLL_20_STD_DF,
                'stats_h2h_cumu_avg'   : TGL_STATS_H2H_CUMU_AVG_DF, 'stats_h2h_cumu_std': TGL_STATS_H2H_CUMU_STD_DF,
            }
            # Extract the team and season from the directory
            TGL_STATS_DIR = SRC_TGL_CSV.replace(SRC_DIR,'').replace('.csv','')
            make_directory(f'{TGT_DIR}/{TGL_STATS_DIR}')
            for TBL_ID, DF in TGL_DICT.items():
                DF.to_csv(f'{TGT_DIR}/{TGL_STATS_DIR}/{TBL_ID}.csv', index=True)
            
        except Exception as e:
            print(f'Error computing stats for {SRC_TGL_CSV}: {e}')
            _FAILS_.append(SRC_TGL_CSV)
            continue
    if _FAILS_:
        print(f'Failed to compute stats for {len(_FAILS_)}/{len(SRC_TGL_CSV_LIST)} files: {_FAILS_}')
    else:
        print(f'All {len(SRC_TGL_CSV_LIST)} files computed successfully')

if __name__ == '__main__':
    if MODE == 'gamelogs_stats':
        compute_gamelog_stats_regular_season()
    else:
        raise ValueError(f'Invalid mode: {MODE}')

    
